In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scripts.ml_method import *
from scripts.proj1_helpers import *
from scripts.preprocess import *
from scripts.split_data import *
from scripts.model_testing import *
from scripts.cross_validation import *

%load_ext autoreload
%autoreload 2

In [15]:
# load data
y, x, ind = load_csv_data('higgs-data/train.csv')

In [16]:
def getPara(x, y):
    for i in range(4):
        mask = x[:, 22] == i
        x_tmp = x[mask, :]
        y_tmp = y[mask]

        if i == 0:
            x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
        elif i == 1:
            x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,26,27,28],1)
        else:
            x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)

        stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)

        losses, w = reg_logistic_regression(y_tmp, stand_x, np.zeros(stand_x.shape[1]), 5000, 0.000002, 0)
        print(np.mean(predict_labels(w, stand_x) == y_tmp))
        
#         k_fold = 5
#         acc_tr = np.zeros(k_fold)
#         acc_te = np.zeros(k_fold)
#         weights = np.zeros([k_fold, stand_x.shape[1]])

#         k_indices = build_k_indices(y_tmp, k_fold, 1)

#         for k in range(k_fold):
#             acc_tr[k], acc_te[k], weights[k,:], losses = cross_validation(y_tmp, stand_x, k_indices, k, 2000, 0)
#             print(acc_tr[k], ', ', acc_te[k], ', ', losses)

        if i == 0:
            w0 = w
#             w0 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w0[j] = sum(weights[:,j])/k_fold
        if i == 1:
            w1 = w
#             w1 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w1[j] = sum(weights[:,j])/k_fold
        if i == 2:
            w2 = w
#             w2 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w2[j] = sum(weights[:,j])/k_fold
        if i == 3:
            w3 = w
#             w3 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w3[j] = sum(weights[:,j])/k_fold
    return w0, w1, w2, w3

w0, w1, w2, w3 = getPara(x, y)

0.839770600422
0.78174971629
0.816471148693


KeyboardInterrupt: 

In [10]:
y, x, ind = load_csv_data('higgs-data/test.csv')

y_pred = np.ones(len(y))

mask = x[:, 22] == 0
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w0, stand_x)

mask = x[:, 22] == 1
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w1, stand_x)

mask = x[:, 22] == 2
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w2, stand_x)

mask = x[:, 22] == 3
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w3, stand_x)

create_csv_submission(ind, y_pred, 'prediction.csv')

## ------ below is just used to test -----

In [134]:
y1 = y*1
x_tr, x_te, y_tr, y_te = split_data(stand_x, y1, 0.8, myseed=1)

for lam in range(100):
    weights = ridge_regression(y_tr, x_tr, lam * 0.01)
    y_pred =  predict_labels(weights, x_te)
    print(np.mean(y_pred == y_te))

0.774389064414
0.759043136243
0.757753562448
0.755367850925
0.755367850925
0.754723064027
0.754271713199
0.753562447611
0.752917660713
0.752530788574
0.751886001676
0.751950480366
0.751950480366
0.750983300019
0.75091882133
0.751047778709
0.750467470501
0.750016119672
0.749951640983
0.749887162293
0.749951640983
0.749500290154
0.749113418015
0.749306854085
0.749435811464
0.749177896705
0.748855503256
0.748597588497
0.748597588497
0.748146237668
0.748146237668
0.747823844219
0.747501450771
0.747050099942
0.746598749113
0.746276355664
0.745889483526
0.745438132697
0.745180217938
0.74466438842
0.744728867109
0.74453543104
0.744213037591
0.74453543104
0.743632729383
0.743374814624
0.742987942485
0.742665549036
0.742601070346
0.742020762138
0.741827326069
0.74131149655
0.740731188342
0.740602230963
0.740666709652
0.740215358824
0.740086401444
0.739635050616
0.739312657167
0.739119221097
0.738667870269
0.73834547682
0.73821651944
0.737958604681
0.737894125991
0.737829647302
0.737507253853
0.